In [1]:
import IPython.display as ipd
import librosa
import numpy as np
import time

from keras.models import Sequential, load_model
from keras.layers import Embedding, Dense, Conv1D
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Adam
from keras.callbacks import Callback

c:\users\troy\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# single channel
dataset_path = 'C:/Datasets/LJSpeech-1.1'
with open('{0}/metadata.csv'.format(dataset_path), encoding='utf-8') as f:
    metadata_str = f.read()
metadata = [line.split('|') for line in metadata_str.split('\n')]
metadata = [ex[:2] for ex in metadata[:-1]]
print('first example:')
ipd.display(ipd.Audio('{0}/wavs/{1}.wav'.format(
    dataset_path, metadata[0][0])))
print(metadata[0][1])

first example:


Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition


In [3]:
print(len(metadata))
print(metadata[0][0], metadata[-1][0])

13100
LJ001-0001 LJ050-0278


In [4]:
y, sr = librosa.load('{0}/wavs/{1}.wav'.format(
    dataset_path, metadata[0][0]), sr=8000)
print(len(y), sr)
ipd.display(ipd.Audio(y, rate=sr))

77241 8000


In [5]:
#librosa.output.write_wav('test.wav', y, sr)

In [6]:
def mu_law(x, mu=255):
    return np.sign(x)*np.log(1 + mu*np.abs(x))/np.log(1 + mu)

def mu_law_inverse(x, mu=255):
    return np.sign(x)*(1/mu)*(np.power(1 + mu, np.abs(x)) - 1)

def pred_sample(prob_vec):
    psum = np.sum(prob_vec)
    choice = np.random.rand()
    itersum = 0
    for idx, val in enumerate(prob_vec):
        itersum += val/psum
        if choice <= itersum:
            return idx
    return 0

In [7]:
def process(x):
    x = mu_law(x)
    x = ((x+1)*128).astype('int32')
    x = np.asarray([255 if x == 256 else x for x in x])
    return x

def deprocess(x):
    x = (x + 0.5)/128. - 1
    x = mu_law_inverse(x)
    return x

In [8]:
yp = process(y)
ydp = deprocess(yp)
ipd.display(ipd.Audio(ydp, rate=sr))

In [31]:
num_groups = 1
num_layers = 9
num_filters = 32
batch_size = 32
batch_len = 2**num_layers

In [32]:
def generate_data():
    while True:
        batch_x = np.zeros((batch_size, batch_len)).astype('int32')
        batch_y = np.zeros((batch_size, batch_len, 256)).astype('int32')
        for batch_idx in range(batch_size):
            data = np.zeros((batch_len+1,))
            dataset_idx = np.random.randint(len(metadata))
            x_aud, _ = librosa.load('{0}/wavs/{1}.wav'.format(
                dataset_path, metadata[dataset_idx][0]), sr=8000)
            slice_s = max(0, len(x_aud) - (batch_len+1))
            if slice_s != 0:
                slice_s = np.random.randint(0, slice_s)
            slice_e = min(slice_s + batch_len+1, len(x_aud))
            slice_len = slice_e - slice_s
            data[:slice_len] = x_aud[slice_s:slice_e]
            data = process(data)
            batch_x[batch_idx][:slice_len-1] = data[:slice_len-1]
            y_slice = data[1:slice_len]
            for j, sample_id in enumerate(y_slice):
                batch_y[batch_idx, j, sample_id] = 1
        yield (batch_x, batch_y)

In [33]:
bx, by = next(generate_data())

In [34]:
print(bx[0,:5])
print(np.argmax(by[0,:5], axis=-1))

[ 26  31  48 138 195]
[ 31  48 138 195 194]


In [35]:
model = Sequential()
model.add(Embedding(input_dim=256, output_dim=num_filters,
    input_length=batch_len))
for i in range(num_groups):
    for j in range(num_layers):
        model.add(Conv1D(filters=num_filters, kernel_size=2,
            dilation_rate=2**j, padding='causal', activation='tanh'))
model.add(TimeDistributed(Dense(256, activation='softmax')))

adam = Adam(0.002)
model.compile(optimizer=adam, loss='categorical_crossentropy')

In [41]:
def generate_audio(samples=24000, samp_id=0):
    bx, by = next(generate_data())
    x = bx[0]
    audio = []
    # todo: fix inefficient sampling
    for i in range(samples):
        y = model.predict(np.array([x]))
        y = pred_sample(y[0, -1])
        x = x[:][1:]
        x = np.concatenate((x, [y]), axis=-1)
        audio.append(deprocess(y))
    audio = np.array(audio)
    fpath = 'outputs/wavenet_{0}_{1}.wav'.format(
        samp_id, int(time.time()))
    librosa.output.write_wav(fpath, audio, 8000)
    print(fpath)
    ipd.display(ipd.Audio(audio, rate=8000))

In [37]:
time_s = time.time()
generate_audio()
print('generated 3 seconds in {0:.2f} seconds'.format(time.time()-time_s))

outputs/output_0_1523334835.wav


generated 3 seconds in 106.73 seconds


In [38]:
class DoGenCB(Callback):
    def __init__(self):
        super(DoGenCB, self).__init__()
    
    def on_epoch_end(self, epoch, logs=None):
        if epoch % 10 == 0:
            generate_audio(samp_id=epoch)

In [43]:
def trainfor(epochs, do_gen=True):
    callbacks = []
    if do_gen:
        callbacks.append(DoGenCB())
    model.fit_generator(generate_data(), 1e2, epochs, callbacks=callbacks)

In [44]:
trainfor(1e9)

Epoch 1/1000000000
 99/100 [============================>.] - ETA: 6s - loss: 4.7855 outputs/wavenet_0_1523336211.wav


100/100 [==============================] - 804s - loss: 4.7806   
Epoch 2/1000000000
100/100 [==============================] - 622s - loss: 4.3017   0
Epoch 3/1000000000
100/100 [==============================] - 748s - loss: 4.2076   2
Epoch 4/1000000000
100/100 [==============================] - 455s - loss: 4.1564   2
Epoch 5/1000000000
100/100 [==============================] - 371s - loss: 4.0710   8
Epoch 6/1000000000
100/100 [==============================] - 349s - loss: 4.0257   5
Epoch 7/1000000000
100/100 [==============================] - 348s - loss: 3.9838   1
Epoch 8/1000000000
100/100 [==============================] - 353s - loss: 3.9431   8
Epoch 9/1000000000
100/100 [==============================] - 328s - loss: 3.8804   6
Epoch 10/1000000000
100/100 [==============================] - 343s - loss: 3.8374   3
Epoch 11/1000000000
 99/100 [============================>.] - ETA: 3s - loss: 3.8126outputs/wavenet_10_1523340533.wav


100/100 [==============================] - 400s - loss: 3.8135   
Epoch 12/1000000000
100/100 [==============================] - 298s - loss: 3.7760   4
Epoch 13/1000000000
100/100 [==============================] - 503s - loss: 3.7357   8
Epoch 14/1000000000
100/100 [==============================] - 407s - loss: 3.6951   8
Epoch 15/1000000000
100/100 [==============================] - 331s - loss: 3.6501   9
Epoch 16/1000000000
100/100 [==============================] - 322s - loss: 3.6448   0
Epoch 17/1000000000
100/100 [==============================] - 325s - loss: 3.6255   5
Epoch 18/1000000000
100/100 [==============================] - 325s - loss: 3.6059   7
Epoch 19/1000000000
100/100 [==============================] - 860s - loss: 3.5967   9
Epoch 20/1000000000
100/100 [==============================] - 321s - loss: 3.5455   2
Epoch 21/1000000000
 99/100 [============================>.] - ETA: 3s - loss: 3.5516outputs/wavenet_20_1523344619.wav


100/100 [==============================] - 389s - loss: 3.5528   
Epoch 22/1000000000
100/100 [==============================] - 288s - loss: 3.5413   1
Epoch 23/1000000000
100/100 [==============================] - 320s - loss: 3.5183   5
Epoch 24/1000000000
100/100 [==============================] - 328s - loss: 3.5357   5
Epoch 25/1000000000
100/100 [==============================] - 589s - loss: 3.5257   5
Epoch 26/1000000000
100/100 [==============================] - 628s - loss: 3.5102   9
Epoch 27/1000000000
100/100 [==============================] - 321s - loss: 3.4896   2
Epoch 28/1000000000
100/100 [==============================] - 330s - loss: 3.4949   6
Epoch 29/1000000000
100/100 [==============================] - 330s - loss: 3.4857   9
Epoch 30/1000000000
100/100 [==============================] - 318s - loss: 3.5004   4
Epoch 31/1000000000
 99/100 [============================>.] - ETA: 3s - loss: 3.4860outputs/wavenet_30_1523348463.wav


100/100 [==============================] - 388s - loss: 3.4851   
Epoch 32/1000000000
100/100 [==============================] - 287s - loss: 3.4542   7
Epoch 33/1000000000
100/100 [==============================] - 447s - loss: 3.4555   5
Epoch 34/1000000000
100/100 [==============================] - 322s - loss: 3.4731   2
Epoch 35/1000000000
100/100 [==============================] - 317s - loss: 3.4366   9
Epoch 36/1000000000
100/100 [==============================] - 319s - loss: 3.4407   4
Epoch 37/1000000000
100/100 [==============================] - 317s - loss: 3.4480   7
Epoch 38/1000000000
100/100 [==============================] - 324s - loss: 3.4276   2
Epoch 39/1000000000
100/100 [==============================] - 325s - loss: 3.4345   9
Epoch 40/1000000000
100/100 [==============================] - 321s - loss: 3.4230   8
Epoch 41/1000000000
 99/100 [============================>.] - ETA: 3s - loss: 3.4172outputs/wavenet_40_1523351832.wav


...

100/100 [==============================] - 392s - loss: 3.2492   
Epoch 122/1000000000
100/100 [==============================] - 285s - loss: 3.2723   5
Epoch 123/1000000000
100/100 [==============================] - 340s - loss: 3.2607   9
Epoch 124/1000000000
100/100 [==============================] - 330s - loss: 3.2620   7
Epoch 125/1000000000
100/100 [==============================] - 327s - loss: 3.2714   6
Epoch 126/1000000000
100/100 [==============================] - 339s - loss: 3.2588   3
Epoch 127/1000000000
100/100 [==============================] - 341s - loss: 3.2576   0
Epoch 128/1000000000
100/100 [==============================] - 341s - loss: 3.2497   2
Epoch 129/1000000000
100/100 [==============================] - 317s - loss: 3.2296   4
Epoch 130/1000000000
100/100 [==============================] - 315s - loss: 3.2431   8
Epoch 131/1000000000
 99/100 [============================>.] - ETA: 3s - loss: 3.2359outputs/wavenet_130_1523382504.wav


100/100 [==============================] - 382s - loss: 3.2361   
Epoch 132/1000000000
100/100 [==============================] - 295s - loss: 3.2668   2
Epoch 133/1000000000
100/100 [==============================] - 312s - loss: 3.2552   9
Epoch 134/1000000000
100/100 [==============================] - 337s - loss: 3.2308   8
Epoch 135/1000000000
100/100 [==============================] - 327s - loss: 3.2237   8
Epoch 136/1000000000
100/100 [==============================] - 331s - loss: 3.2295   8
Epoch 137/1000000000
100/100 [==============================] - 326s - loss: 3.2440   1
Epoch 138/1000000000
100/100 [==============================] - 333s - loss: 3.2421   5
Epoch 139/1000000000
100/100 [==============================] - 345s - loss: 3.2490   8
Epoch 140/1000000000
100/100 [==============================] - 333s - loss: 3.2437   9
Epoch 141/1000000000
 99/100 [============================>.] - ETA: 3s - loss: 3.2474outputs/wavenet_140_1523385837.wav


100/100 [==============================] - 390s - loss: 3.2456   
Epoch 142/1000000000
100/100 [==============================] - 298s - loss: 3.2349   9
Epoch 143/1000000000
100/100 [==============================] - 328s - loss: 3.2463   1
Epoch 144/1000000000
100/100 [==============================] - 341s - loss: 3.2366   2
Epoch 145/1000000000
100/100 [==============================] - 372s - loss: 3.2224   0
Epoch 146/1000000000
100/100 [==============================] - 345s - loss: 3.2481   4
Epoch 147/1000000000
100/100 [==============================] - 330s - loss: 3.2304   6
Epoch 148/1000000000
100/100 [==============================] - 319s - loss: 3.2211   5
Epoch 149/1000000000
100/100 [==============================] - 313s - loss: 3.2171   2
Epoch 150/1000000000
100/100 [==============================] - 330s - loss: 3.2323   9
Epoch 151/1000000000
 99/100 [============================>.] - ETA: 3s - loss: 3.2552outputs/wavenet_150_1523389198.wav


100/100 [==============================] - 381s - loss: 3.2541   
Epoch 152/1000000000
 14/100 [===>..........................] - ETA: 80s - loss: 3.2226

KeyboardInterrupt: 

In [45]:
model.save('models/wn_e=150_l=3.22.h5')